In [1]:
#Importing used libraries.

import numpy as np
import matplotlib.pyplot as plt
from scipy.integrate import solve_ivp
import time
import pandas as pd
from IPython.display import display, Markdown, HTML
import pandas as pd

In [ ]:
#Defining used functions

def U(x):
    return 1/(100 * np.sqrt(x[0]**2 + x[1]**2))

def U_tokamak(x):
    return 0

def grad_U(x):
    return -1/100 * np.array([x[0]/(x[0]**2 + x[1]**2)**(3/2),x[1]/(x[0]**2 + x[1]**2)**(3/2),0.0])

def grad_U_tokamak(x):
    return np.array([0,0,0])

def B_tokamak(x):
    return np.array([-(2*x[1] + x[0]*x[2])/(2*(x[0]**2+x[1]**2)),
                     (2*x[0]-x[1]*x[2])/(2*(x[0]**2 + x[1]**2)),
                     (np.sqrt(x[0]**2 + x[1]**2)-1)/(2*np.sqrt(x[0]**2 + x[1]**2))])

def B(x):
    return np.array([0.0,0.0,np.sqrt(x[0]**2 + x[1]**2)])

def A(x):
    return 1/3 * np.array([-x[1]*np.sqrt(x[0]**2 + x[1]**2),
                           x[0]*np.sqrt(x[0]**2 + x[1]**2),
                           0.0])

def A_tokamak(x):
    return np.array([0.0,
                     1/2*(x[0]-np.log(np.sqrt(x[0]**2+x[1]**2)+x[0])),
                     -1/2*(x[2]*np.arctan2(x[1],x[0])+np.log(x[0]**2+x[1]**2))])

def energy(x,v,U):
    return 1/2 * (np.linalg.norm(v))**2 + U(x)

def jacobian_tokamak(xs):
    x, y, z = xs
    eps = 1.0e-12 
    r2 = x*x + y*y
    r2 = np.clip(r2, eps**2, None)  
    r  = np.sqrt(r2)

    f2x1 = 0.5 - 0.5 * ((x/r + 1.0) / (r + x))
    f2x2 = -0.5 * ((y/r) / (r + x))

    f3x1 = 0.5 * z * (y / r2) - (x / r2)
    f3x2 = -0.5 * z * (x / r2) - (y / r2)
    f3x3 = -0.5 * np.arctan2(y, x)

    J = np.array([
        [0.0, 0.0, 0.0],
        [f2x1, f2x2, 0.0],
        [f3x1, f3x2, f3x3]
    ])
    return J

def jacobian(xs):
    eps=1e-12
    x1, x2, x3 = xs
    r = np.hypot(x1, x2)
    r = max(r, eps)          
    c = 1.0/3.0

    J = np.array([
        [ -c*(x1*x2)/r,          -c*(x1*x1 + 2.0*x2*x2)/r,  0.0 ],
        [  c*(2.0*x1*x1 + x2*x2)/r,  c*(x1*x2)/r,            0.0 ],
        [  0.0,                   0.0,                       0.0 ],
    ], dtype=float)
    return J

def f(x,v):
    return np.array([v,np.cross(v,B(x)) - grad_U(x)])

def f_tokamak(x,v):
    return np.array([v,np.cross(v,B_tokamak(x))])

In [ ]:
#Implementation of RK4 solver

def RK4(x0,v0,h,f,T):

    x = x0
    v = v0

    t = 0

    x_list = [x]
    time_list = [t]
    v_list = [v]

    while t < T:

        k1 = h * f(x,v)
        k2 = h * f(x + 1/2 * k1[0],v + 1/2 * k1[1])
        k3 = h * f(x + 1/2 * k2[0],v + 1/2 * k2[1])
        k4 = h * f(x + k3[0],v + k3[1])

        x = x + 1/6 * (k1[0]+2*k2[0]+2*k3[0]+k4[0])
        v = v + 1/6 * (k1[1]+2*k2[1]+2*k3[1]+k4[1])

        t += h

        x_list.append(x)
        time_list.append(t)
        v_list.append(v)

    return x_list,v_list,time_list

In [ ]:
#Implementation

def boris(x0,v0,h,T,B,grad_U,f_flat):

    sol = solve_ivp(f_flat,(0,-1/2*h),np.concatenate([x0,v0]))

    v = sol.y[3:, -1]
    x = x0

    n = 0

    x_list = [x]
    v_list = []

    while n < T:

        
        t = h/2 * B(x)
        s = 2*t/(1+np.dot(t,t))

        v_minus = v - h/2 * grad_U(x)
        v_prime = v_minus + np.cross(v_minus,t)
        v_plus = v_minus + np.cross(v_prime,s)

        v = v_plus - h/2 * grad_U(x)
        x = x + h*v

        n += h

        x_list.append(x)
        v_list.append(1/2 * (v_plus + v_minus))

    t = h/2 * B(x)
    s = 2*t/(1+np.dot(t,t))

    v_minus = v - h/2 * grad_U(x)
    v_prime = v_minus + np.cross(v_minus,t)
    v_plus = v_minus + np.cross(v_prime,s)

    v_list.append(1/2 * (v_plus + v_minus))

    return x_list,v_list

In [ ]:
# Defining constants used in the symmetric multistep method

a1 = -0.7
a2 = 0.1
a3 = 0.9

beta = 1/3 * np.array([20*a1*a2*a3 - 4*(a1*a2+a1*a3+a2*a3) - 28*(a1+a2+a3) - 52,
                       2*a1*a2*a3 + 14*(a1*a2+a1*a3+a2*a3) + 26*(a1+a2+a3) + 38])

alpha = np.array([1,-7/5,9/25,22/125,-34/125,22/125,9/25,-7/5,1])

In [ ]:
def multistep_method(x0,v0,h,A,T,f_flat,grad_U,jacobian):

    sol = solve_ivp(f_flat,[0,7*h],np.concatenate([x0,v0]),t_eval=np.arange(0,8*h,h),rtol=1e-10,atol=1e-12)
    x_list = list(sol.y[:3,:].T)
    A_list = [A(i) for i in x_list]

    j = 4
    t = 7*h
    F_list = [0,0,0]
    v_list = [0,0]

    v_list.append(1/(12*h) * (x_list[0] - 8*x_list[1] + 8*x_list[3] - x_list[4]))

    for i in range(3,5):
        v = 1/(12*h) * (x_list[i-2] - 8*x_list[i-1] + 8*x_list[i+1] - x_list[i+2])
        A_delta = 1/(12*h) * (A_list[i-2] - 8*A_list[i-1] + 8*A_list[i+1] - A_list[i+2])
        F = jacobian(x_list[i]).T @ v - A_delta - grad_U(x_list[i]) 
        F_list.append(F)
        v_list.append(v)

    v_list.append(1/(12*h) * (x_list[3] - 8*x_list[4] + 8*x_list[6] - x_list[7]))
    
    while t < T:

        v = 1/(12*h) * (x_list[j-1] - 8*x_list[j] + 8*x_list[j+2] - x_list[j+3])        
        A_delta = 1/(12*h) * (A_list[j-1] - 8*A_list[j] + 8*A_list[j+2] - A_list[j+3])
        F = jacobian(x_list[j+1]).T @ v - A_delta - grad_U(x_list[j+1])
        F_list.append(F)
        v_list.append(v)

        summed_x = h**2 * (beta[1]*F_list[j-1] + beta[0]*F_list[j] + beta[1]*F_list[j+1])
        sum_so_far = np.array([0,0,0])
        for i in range(0,8):
            sum_so_far = sum_so_far + alpha[i] * x_list[-(8-i)]
        x = 1/alpha[-1] * (summed_x - sum_so_far)
        x_list.append(x)
        A_list.append(A(x))
        j += 1
        t += h
    
    v_list.append(0)
    v_list.append(0)

    return x_list,v_list

In [ ]:
# This are the same functions as the original f functions defined earlier. Instead of returning a 2D array, they return a 1D array as [x0,x1,x2,v0,v1,v2].
# Scipy solve_ivp requiers this form to work.

def f_flat(t,y):
    x = y[:3]
    v = y[3:]
    return np.concatenate(f(x,v))

def f_flat_tokamak(t,y):
    x = y[:3]
    v = y[3:]
    return np.concatenate(f_tokamak(x,v))

In [ ]:
#Running simulations for the particle in the 2D charged magnetic field

x0 = np.array([0.0,1.0,0.1])
v0 = np.array([0.09,0.05,0.20])
h = 0.1
T = 1e5

x_list_RK4,v_list_RK4,time_list = RK4(x0,v0,h,f,T)
print("RK4 done")
x_list_boris,v_list_boris = boris(x0,v0,h,T,B,grad_U,f_flat)
print("Boris method done")
x_list_multistep,v_list_multistep = multistep_method(x0,v0,h,A,T,f_flat,grad_U,jacobian)
print("Multistep method done")
ref = solve_ivp(f_flat,[0,time_list[-1]],np.concatenate([x0,v0]),t_eval=time_list,atol=1e-12,rtol=1e-10)
print("Reference done")

In [ ]:
# Plotting posistions for the 2D charged particle

fig , ax = plt.subplots(1,3,figsize=(12,4))
ax[0].plot([x[0] for x in x_list_RK4],[x[1] for x in x_list_RK4],color="r")
ax[1].plot([x[0] for x in x_list_boris],[x[1] for x in x_list_boris],color="g")
ax[2].plot([x[0] for x in x_list_multistep],[x[1] for x in x_list_multistep],color="b")
ax[0].set_title("RK4")
ax[1].set_title("Boris method")
ax[2].set_title("Multistep method")
ax[0].set_xlabel("x")
ax[1].set_xlabel("x")
ax[2].set_xlabel("x")
ax[0].set_ylabel("y")
ax[1].set_ylabel("y")
ax[2].set_ylabel("y")
fig.suptitle("Particle position 2D charged particle")
plt.tight_layout()
plt.show()

We can clearly see that in this particular magnetic field the particle moves in a circle. Visually we obtain the same result for all methods. The same result is obtained in Hairer and Lubich (2017) in figure 1 and 2.

In [ ]:
#Running simulations for the particle in the tomamak magnetic field.

x0 = np.array([1.2,0.0,0.0])
v0 = np.array([0.0,4.816e-4,-2.059e-3])
h = 0.1
T = 1e5

x_list_RK4_tok,v_list_RK4_tok,time_list = RK4(x0,v0,h,f_tokamak,T)
print("RK4 done")
x_list_boris_tok,v_list_boris_tok = boris(x0,v0,h,T,B_tokamak,grad_U_tokamak,f_flat_tokamak)
print("Boris method done")
x_list_multistep_tok,v_list_multistep_tok = multistep_method(x0,v0,h,A_tokamak,T,f_flat_tokamak,grad_U_tokamak,jacobian_tokamak)
print("Multistep method done")
ref_tokamak = solve_ivp(f_flat_tokamak,[0,time_list[-1]],np.concatenate([x0,v0]),t_eval=time_list,atol=1e-12,rtol=1e-10)
print("Referance done")

In [ ]:
# Plotting positions for the particle in the tokamak magnetic field

fig , ax = plt.subplots(1,3,figsize=(12,4))
ax[0].plot([x[0] for x in x_list_RK4_tok],[x[1] for x in x_list_RK4_tok],color="r")
ax[1].plot([x[0] for x in x_list_boris_tok],[x[1] for x in x_list_boris_tok],color="g")
ax[2].plot([x[0] for x in x_list_multistep_tok],[x[1] for x in x_list_multistep_tok],color="b")
fig.suptitle("Particle position in Tokamak magnetic field from above")
ax[0].set_title("RK4")
ax[1].set_title("Boris method")
ax[2].set_title("Multistep method")
ax[0].set_xlabel("x")
ax[1].set_xlabel("x")
ax[2].set_xlabel("x")
ax[0].set_ylabel("y")
ax[1].set_ylabel("y")
ax[2].set_ylabel("y")
plt.tight_layout()
plt.show()
fig , ax = plt.subplots(1,3,figsize=(12,4))
ax[0].plot([np.sqrt(x[0]**2 + x[1]**2) for x in x_list_RK4_tok],[x[2] for x in x_list_RK4_tok],color="r")
ax[1].plot([np.sqrt(x[0]**2 + x[1]**2) for x in x_list_boris_tok],[x[2] for x in x_list_boris_tok],color="g")
ax[2].plot([np.sqrt(x[0]**2 + x[1]**2) for x in x_list_multistep_tok],[x[2] for x in x_list_multistep_tok],color="b")
fig.suptitle("Cross section of particle position in Tokamak magnetic field")
ax[0].set_title("RK4")
ax[1].set_title("Boris method")
ax[2].set_title("Multistep method")
ax[0].set_xlabel("R")
ax[1].set_xlabel("R")
ax[2].set_xlabel("R")
ax[0].set_ylabel("z")
ax[1].set_ylabel("z")
ax[2].set_ylabel("z")
plt.tight_layout()
plt.show()

In the case where we have a Tokamak magnetic field, we still obtain equal visual result for each method. From the plots above we can see that the particle seems to be moving in a half circle around the tokamak. Instead of going in full circles like in the 2D magnetic field, the particle seems to oscillate back and forth in this half circle. In the plots below we can see a cross section of the torus shaped tokamak chamber. While moving around the tokamak in a half circle, the particle moves radially in this banana shaped pattern. This was the expected result, and this is also obtained in Hairer and Lubich (2017).

In [ ]:
# Computing position errors

errors_RK4 = [np.linalg.norm(x_list_RK4[i] - ref.y[:3].T[i]) for i in range(len(time_list))]
errors_boris = [np.linalg.norm(x_list_boris[i] - ref.y[:3].T[i]) for i in range(len(time_list))]
errors_multistep = [np.linalg.norm(x_list_multistep[i] - ref.y[:3].T[i]) for i in range(len(time_list))]
errors_RK4_tokamak = [np.linalg.norm(x_list_RK4_tok[i] - ref_tokamak.y[:3].T[i]) for i in range(len(time_list))]
errors_boris_tokamak = [np.linalg.norm(x_list_boris_tok[i] - ref_tokamak.y[:3].T[i]) for i in range(len(time_list))]
errors_multistep_tokamak = [np.linalg.norm(x_list_multistep_tok[i] - ref_tokamak.y[:3].T[i]) for i in range(len(time_list))]

In [ ]:
# Plotting errors

fig , ax = plt.subplots(1,3,figsize=(12,4))
ax[0].semilogy(time_list,errors_RK4,color="r")
ax[1].semilogy(time_list,errors_boris,color="g")
ax[2].semilogy(time_list,errors_multistep,color="b")
fig.suptitle("Errors for 2D charged particle")
ax[0].set_title("RK4")
ax[1].set_title("Boris method")
ax[2].set_title("Multistep method")
ax[0].set_xlabel("t")
ax[1].set_xlabel("t")
ax[2].set_xlabel("t")
ax[0].set_ylabel("Error")
ax[1].set_ylabel("Error")
ax[2].set_ylabel("Error")
plt.tight_layout()
plt.show()
fig , ax = plt.subplots(1,3,figsize=(12,4))
ax[0].semilogy(time_list,errors_RK4_tokamak,color="r")
ax[1].semilogy(time_list,errors_boris_tokamak,color="g")
ax[2].semilogy(time_list,errors_multistep_tokamak,color="b")
fig.suptitle("Errors for particle in Tokamak magnetic field ")
ax[0].set_title("RK4")
ax[1].set_title("Boris method")
ax[2].set_title("Multistep method")
ax[0].set_xlabel("t")
ax[1].set_xlabel("t")
ax[2].set_xlabel("t")
ax[0].set_ylabel("Error")
ax[1].set_ylabel("Error")
ax[2].set_ylabel("Error")
plt.tight_layout()
plt.show()

In the plots above we can observe the errors of the numerical methods against a reference method. This reference method is a RK45 method with very high tolerance. In the case of the particle in the Tokamak magnetic field we can observe that the error of the RK4 seems to oscillate but overall increase. The Boris method and the symmetric multistep method seems to be upper bounded. The periodic oscillations observed is likely due to the approximation lining up with the reference with some time period. This is expected when the particles moves back and forth in the same trajectory.

In [ ]:
# Computing relative energy

energy_RK4 = [np.abs(energy(x_list_RK4[i],v_list_RK4[i],U) - energy(x_list_RK4[0],v_list_RK4[0],U)) for i in range(len(time_list))]
energy_boris = [np.abs(energy(x_list_boris[i],v_list_boris[i],U) - energy(x_list_boris[0],v_list_boris[0],U)) for i in range(len(time_list))]
energy_multistep = [np.abs(energy(x_list_multistep[i],v_list_multistep[i],U) - energy(x_list_multistep[2],v_list_multistep[2],U)) for i in range(2,len(time_list)-2)]
energy_RK4_tokamak = [np.abs(energy(x_list_RK4_tok[i],v_list_RK4_tok[i],U_tokamak) - energy(x_list_RK4_tok[0],v_list_RK4_tok[0],U_tokamak)) for i in range(len(time_list))]
energy_boris_tokamak = [np.abs(energy(x_list_boris_tok[i],v_list_boris_tok[i],U_tokamak) - energy(x_list_boris_tok[0],v_list_boris_tok[0],U_tokamak)) for i in range(len(time_list))]
energy_multistep_tokamak = [np.abs(energy(x_list_multistep_tok[i],v_list_multistep_tok[i],U_tokamak) - energy(x_list_multistep_tok[0],v_list_multistep_tok[0],U_tokamak)) for i in range(2,len(time_list)-2)]

In [ ]:
# Plotting relative energy

fig , ax = plt.subplots(1,3,figsize=(12,4))
ax[0].semilogy(time_list,energy_RK4,color="r")
ax[1].semilogy(time_list,energy_boris,color="g")
ax[2].semilogy(time_list[2:-2],energy_multistep,color="b")
fig.suptitle("Relative energy for 2D charged particle")
ax[0].set_title("RK4")
ax[1].set_title("Boris method")
ax[2].set_title("Multistep method")
ax[0].set_xlabel("t")
ax[1].set_xlabel("t")
ax[2].set_xlabel("t")
ax[0].set_ylabel("Relative energy")
ax[1].set_ylabel("Relative energy")
ax[2].set_ylabel("Relative energy")
plt.tight_layout()
plt.show()
fig , ax = plt.subplots(1,3,figsize=(12,4))
ax[0].semilogy(time_list,energy_RK4_tokamak,color="r")
ax[1].semilogy(time_list,energy_boris_tokamak,color="g")
ax[2].semilogy(time_list[2:-2],energy_multistep_tokamak,color="b")
fig.suptitle("Relative energy for particle in Tokamak magnetic field")
ax[0].set_title("RK4")
ax[1].set_title("Boris method")
ax[2].set_title("Multistep method")
ax[0].set_xlabel("t")
ax[1].set_xlabel("t")
ax[2].set_xlabel("t")
ax[0].set_ylabel("Relative energy")
ax[1].set_ylabel("Relative energy")
ax[2].set_ylabel("Relative energy")
plt.tight_layout()
plt.show()

We see that the relative energy increases for the RK4 method. This is because energy is not conserved by the RK4 method. For the boris and multistep method the relative energy seems to be bounded and thus the energy is conserved. The results seems to be correct with respect to theorem 3.2 in Hairer and Lubich (2017). The theorem states that for s-stabel symmetric multistep methods the total energy is conserved with an relative energy $\Delta E = \mathcal{O}(h^{p})$ up to $\mathcal{O}(h^{-p-2})$ number of timesteps. The symmetric multistep method is a s-stable and of order 4. With h = 0.1 the relative energy of the symmetic multistep method is supposed to be $\Delta E \leq 10^{-4}$. We observe that $\Delta E \leq 10^{-4}$ for the particle in the 2D system and $\Delta E \approx 10^{-6}$ for the particle in the tokamak system. Experimentally the theorem seems to be correct. The relative energy is also ocillating with the same pattern obtained in Hairer and Lubich (2017) in figure 5.1. Even though the boris method is of lower order it seems to conserve the energy better with $\Delta E \leq 10^{-5}$ for the particle in the 2D system and $\Delta E \leq 10^{-10}$ for the particle in the tokamak system.